potato dis

In [60]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris


In [3]:
import tensorflow as tf
IMAGE_SIZE=256
BATCH_SIZE=32
CHANNELS=3
EPOCHS=50

dataset=tf.keras.preprocessing.image_dataset_from_directory(
    "PlantVillage",
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE, 

 )

Found 2158 files belonging to 4 classes.


In [4]:
class_names= dataset.class_names
print(class_names)

len(dataset)
print(len(dataset))
 

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'my_env']
68


In [5]:
for image_batch, label_batch in dataset.take(1):  
    plt.figure(figsize=(12,9))
     
for i in range(12): 
    ax=plt.subplot(3,4,i+1) 
    plt.imshow(image_batch[i].numpy().astype("uint8"))
    plt.title(class_names[label_batch[i]])
    
    plt.axis("off")
    plt.show()

In [6]:
train_size =0.8
len(dataset)*train_size

dataset.take(54)

train_ds=dataset.take(54)
print(train_ds)


test_ds=dataset.skip(54)
len(test_ds)

val_ds=dataset.skip(54)
len(test_ds)

test_ds=test_ds.skip(6)
len(test_ds)

<_TakeDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


8

In [7]:
def get_dataset_partitions_tf(ds,train_split=0.8,val_split=0.1,shuffle=True,shuffle_size=100000):
    ds_size=len(ds)

    if shuffle:
       ds= ds.shuffle(shuffle_size,seed=12)

    train_size=int(train_split* ds_size) 
    val_size=int(val_split*ds_size)
    train_ds=ds.take(train_size)

    val_ds=ds.skip(train_size).take(val_size)
    test_ds=ds.skip(train_size).skip(val_size)

    return train_ds,val_ds,test_ds


In [8]:
train_ds,val_ds,test_ds=get_dataset_partitions_tf(dataset)

len(train_ds)
print(len(train_ds))

54


In [9]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

resize_and_rescale = tf.keras.Sequential([
    tf.keras.layers.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    tf.keras.layers.Rescaling(1.0/255),   
])

data_augmentation=tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),
]) 

In [10]:
IMAGE_SIZE = 256  # Example image size
CHANNELS = 3  # For RGB images
n_classes = 3

# Define the model
model = tf.keras.Sequential([
    tf.keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS)),  # Define input shape here
    resize_and_rescale,  # Rescaling and resizing layers
    data_augmentation,   # Data augmentation layers
    layers.Conv2D(32, (3, 3), activation="relu"),  # No input_shape here
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(n_classes, activation="softmax"),
])

# No need for model.build(input_shape=input_shape)
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 12, 12, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 4, 4, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 2, 2, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 183,747 (717.76 KB)

 Trainable params: 183,747 (717.76 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
IMAGE_SIZE = 256  # Example image size
CHANNELS = 3  # For RGB images
n_classes = 4

# Define the model
model = tf.keras.Sequential([
    tf.keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS)),  # Define input shape here
    resize_and_rescale,  # Rescaling and resizing layers
    data_augmentation,   # Data augmentation layers
    layers.Conv2D(32, (3, 3), activation="relu"),  # No input_shape here
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(n_classes, activation="softmax"),
])

# No need for model.build(input_shape=input_shape)
model.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 60, 60, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 12, 12, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 4, 4, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 2, 2, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 183,812 (718.02 KB)

 Trainable params: 183,812 (718.02 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"],  
    )

In [32]:
plt.figure(figsize=((8,8)))
plt.subplot(1,2,1)
plt.plot(range(EPOCHS),acc,label="Training Accuracy")
plt.plot(range(EPOCHS), val_acc,label="Validation Accuracy")
plt.legend(loc="lower right")
plt.title("Training and Validation Accuracy")

plt.subplot(1,2,2)
plt.plot(range(EPOCHS),loss,label="Training loss")
plt.plot(range(EPOCHS),val_loss,label="Validation Loss")
plt.legend(loc="upper right")
plt.title("Training and Validation loss")
plt.show()



In [24]:
histroy=model.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1
)
#Save the entire model
model.save('my_model.keras')

Epoch 1/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 232s 4s/step - accuracy: 0.9941 - loss: 0.0275 - val_accuracy: 0.9688 - val_loss: 0.0572
Epoch 2/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 381s 7s/step - accuracy: 0.9890 - loss: 0.0367 - val_accuracy: 0.9375 - val_loss: 0.2311
Epoch 3/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 288s 5s/step - accuracy: 0.9658 - loss: 0.1030 - val_accuracy: 0.9844 - val_loss: 0.0327
Epoch 4/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 227s 4s/step - accuracy: 0.9758 - loss: 0.0646 - val_accuracy: 0.9740 - val_loss: 0.0429
Epoch 5/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 319s 6s/step - accuracy: 0.9914 - loss: 0.0221 - val_accuracy: 0.9688 - val_loss: 0.0895
Epoch 6/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 172s 3s/step - accuracy: 0.9802 - loss: 0.0463 - val_accuracy: 0.9948 - val_loss: 0.0149
Epoch 7/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 151s 3s/step - accuracy: 0.9948 - loss: 0.0154 - val_accuracy: 0.9948 - val_loss: 0.0340
Epoch 8/50
54/54 ━━━━━━━━━━━━━━━━━━━━ 151s 3s/step - accuracy: 0.9934 - loss: 0.0263 - val_accuracy: 0.9740 - v

In [22]:
model.save('my_model.keras')

In [30]:
acc=histroy.history["accuracy"]
val_acc=histroy.history["val_accuracy"]

loss=histroy.history["loss"]
val_loss=histroy.history["val_loss"]

In [47]:
def predict(model,img):
    img_array=tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array=tf.expand_dims(img_array,0)

    predictions=model.predict(img_array)

    predicted_class =class_names[np.argmax(predictions[0])]
    confidence=round(100 *(np.max(predictions[0])),2)
    return predicted_class,confidence

In [26]:
scores=model.evaluate(test_ds)

8/8 ━━━━━━━━━━━━━━━━━━━━ 26s 3s/step - accuracy: 0.9673 - loss: 0.0980


In [51]:
plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):
    for i in range(9):
        # Create a subplot (3x3 grid)
        ax = plt.subplot(3, 3, i + 1)

        # Display the image
        plt.imshow(images[i].numpy().astype("uint8"))

        # Predict the class and confidence for the current image
        predicted_class, confidence = predict(model, images[i].numpy())

        # Get the actual class from labels
        actual_class = class_names[labels[i]]  # Fixed the indexing here

        # Set the title with predicted and actual information
        plt.title(f"Actual: {actual_class},\nPredicted: {predicted_class},\nConfidence: {confidence}%")

        # Turn off axis
        plt.axis("off")

# Show all the plots
plt.show()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step


In [27]:
scores

[0.07321549952030182, 0.97265625]

In [28]:
history

plt.figure(figsize=((8,8)))
plt.subplot(1,2,1)
plt.plot(range(EPOCHS),acc,label="Training Accuracy")
plt.plot(range(EPOCHS), val_acc,label="Validation Accuracy")
plt.legend(loc="lower right")
plt.title("Training and Validation Accuracy")

plt.subplot(1,2,2)
plt.plot(range(EPOCHS),loss,label-"Training loss")
plt.plot(range(EPOCHS),val_loss,label="Validation Loss")
plt.legend(loc="upper right")
plt.title("Training and Validation loss")
plt.show()
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import tensorflow as tf
IMAGE_SIZE=256
BATCH_SIZE=32
CHANNELS=3
EPOCHS=50

dataset=tf.keras.preprocessing.image_dataset_from_directory(
    "PlantVillage",
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE, 

 )
class_names= dataset.class_names
print(class_names)


In [36]:
import numpy as np
for images_batch,labels_batch in test_ds.take(1):
    first_image= images_batch[0].numpy().astype("uint8")
    first_label=labels_batch[0].numpy()

    print("first image to predict")
    plt.imshow(first_image)
    print("actual label:",class_names[first_label])
    
    batch_prediction=model.predict(images_batch)
    print("predicted label:",class_names[np.argmax(batch_prediction[0])])
    plt.show()

first image to predict
actual label: Potato___Early_blight
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
predicted label: Potato___Early_blight


In [ ]:
np.argmax

In [29]:
# Accessing the history attribute from the model
history = model.history

# Now you can access history parameters
print(history.params)


{'verbose': 'auto', 'epochs': 1, 'steps': 8}


In [2]:
import os

model_version = 1
folder_path = "../models"

# Create the folder if it doesn't exist
os.makedirs(folder_path, exist_ok=True)


In [11]:
import os
print(os.getcwd())


c:\Users\User\Downloads\open cv1


In [6]:
model_version = 2
folder_path = "../models"
os.makedirs(folder_path, exist_ok=True)


In [8]:
folder_path = "../models"


In [9]:
import os

# Get the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# List contents of the parent directory
contents = os.listdir(parent_dir)
print(contents)


['03-111-100-235.jpg', '03-111-100-235.mp4', '0400002686390_420016755571.pdf', '0400002686390_440017334806.pdf', '0400019430737_620014877392.pdf', '1.jpg', '120.mp4', '2.jpg', '21-AGREEMENT OF SALE (SPECIMEN for Talha)  17-01-2024 (1).doc', '21-AGREEMENT OF SALE (SPECIMEN for Talha)  17-01-2024.doc', '3.jpg', '344862666_1519512131912764_4301515675191701211_n.jpg', '345457477_249968891044032_2123190336957630655_n.jpg', '346657943_807294934164312_6268019518334069826_n (1).jpg', '346657943_807294934164312_6268019518334069826_n.jpg', '347444699_636805125147320_6486426568715487641_n.jpg', '355659578_641516071342892_4144501110656756565_n.jpg', '358364730_649305687230597_8088635103377100241_n.jpg', '363378434_664591789035320_1904662749989858662_n.jpg', '363419473_661159819378517_4611334001721020685_n.jpg', '386107807_708116901349475_4277097889077288383_n.jpg', '396718767_716389470522218_5376700332139840292_n.jpg', '397893909_718043583690140_4112685476408118089_n.jpg', '400064051_7202092701402

In [13]:
import os


model_version=2
folder_path = "../modelsss"
os.makedirs(folder_path, exist_ok=True)  # This won't throw an error if it already exists


In [14]:
import pickle
import os

folder_path = "../models"
os.makedirs(folder_path, exist_ok=True)

class MyModel:
    def __init__(self):
        self.parameters = [1, 2, 3]

model = MyModel()  # Make sure the model is instantiated

# Save the model
with open(f"{folder_path}/model.pkl", "wb") as file:
    pickle.dump(model, file)
